In [22]:
import cv2
import csv
import keras 
from keras.layers import Dense, GlobalAveragePooling2D
from keras.applications.densenet import DenseNet121
import numpy as np
from keras.models import Model
from sklearn.model_selection import train_test_split

In [2]:
# read in and preprocess image
def readImage(pathname):
    oriim = cv2.imread(pathname)
    if oriim is None:
        return None
    im = cv2.resize(oriim, (224, 224)).astype(np.float32)
    # Subtract mean pixel and multiple by scaling constant 
    # Reference: https://github.com/shicai/DenseNet-Caffe
    im[:,:,0] = (im[:,:,0] - 103.94) * 0.017
    im[:,:,1] = (im[:,:,1] - 116.78) * 0.017
    im[:,:,2] = (im[:,:,2] - 123.68) * 0.017
    return im

In [39]:
# load train data
directory = '../data/images/'

# load annotation into memory first
annotations = {}
with open('../data/annotations.txt', 'r+') as csvfile:
    csvreader = csv.reader(csvfile, delimiter = ',')
    for row in csvreader:
        annotations[row[0]] = int(row[1])-1
        #if not row[0] in annotations:

size = len(annotations)
print(size)

y_data = np.zeros((size,1), dtype=np.int)
x_data = np.zeros((size,224,224,3), dtype=np.float32)

count = 0
for key,value in annotations.items():
    im = readImage(directory+key+'.jpg')
    # handle error
    if im is None:
        continue
    im = np.expand_dims(im, axis=0)
    x_data[count] = im

    label = value
    label_np = np.array([label])
    label_np = np.expand_dims(label_np, axis=0)
    y_data[count] = label_np
    
    count += 1

print(count)
print('x_data shape:', x_data.shape)
print('y_data shape:', y_data.shape)

count -= 1
x_data = x_data[:count]
y_data = y_data[:count]
print('x_data shape:', x_data.shape)
print('y_data shape:', y_data.shape)

194828
1183
x_data shape: (194828, 224, 224, 3)
y_data shape: (194828, 1)
x_data shape: (1182, 224, 224, 3)
y_data shape: (1182, 1)


In [14]:
# create the base pre-trained model
dense_model = keras.applications.densenet.DenseNet121(weights='imagenet', include_top=False)
dense_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, None, None, 6 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, None, None, 6 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [15]:
# add a global spatial average pooling layer
x = dense_model.output
x = GlobalAveragePooling2D()(x)

In [16]:
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)

In [17]:
# and a logistic layer -- let's say we have 128 classes
predictions = Dense(128, activation='softmax')(x)

In [18]:
# this is the model we will train
my_model = Model(inputs=dense_model.input, outputs=predictions)


In [10]:
my_model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, None, None, 6 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, None, None, 6 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

conv4_block17_concat (Concatena (None, None, None, 8 0           conv4_block16_concat[0][0]       
                                                                 conv4_block17_2_conv[0][0]       
__________________________________________________________________________________________________
conv4_block18_0_bn (BatchNormal (None, None, None, 8 3200        conv4_block17_concat[0][0]       
__________________________________________________________________________________________________
conv4_block18_0_relu (Activatio (None, None, None, 8 0           conv4_block18_0_bn[0][0]         
__________________________________________________________________________________________________
conv4_block18_1_conv (Conv2D)   (None, None, None, 1 102400      conv4_block18_0_relu[0][0]       
__________________________________________________________________________________________________
conv4_block18_1_bn (BatchNormal (None, None, None, 1 512         conv4_block18_1_conv[0][0]       
__________

In [19]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in dense_model.layers:
    layer.trainable = False
#my_model.summary() # trainable params decrease

In [ ]:
my_model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [41]:
X_train, X_test, y_train, y_test = train_test_split(
...     x_data, y_data, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)

(945, 224, 224, 3)
(237, 224, 224, 3)


In [42]:
y_train_binary = keras.utils.np_utils.to_categorical(y_train)
y_test_binary = keras.utils.np_utils.to_categorical(y_test)
print(y_train_binary.shape)
print(y_test_binary.shape)
print(y_train.shape)

(945, 128)
(237, 128)
(945, 1)
(1182, 128)


In [ ]:
# train the model on the new data for a few epochs
batch_size = 32
nb_epoch = 200
result=my_model.fit(X_train, y_train_binary,
                  batch_size=batch_size,
                  nb_epoch=nb_epoch,
                  validation_data=(X_test, y_test_binary),
                  shuffle=True)

/Users/zhuqishan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 945 samples, validate on 237 samples
Epoch 1/200
945/945 [==============================] - 429s 454ms/step - loss: 4.2693 - val_loss: 3.5000
Epoch 2/200
 32/945 [>.............................] - ETA: 5:37 - loss: 1.2984